In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from gpflow.kernels import Matern32
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt

pd.set_option('display.float_format', lambda x: '%.6f' % x)

def fetch_and_process_data(ticker, start_date="2024-01-01", end_date="2024-12-31", train_ratio=0.8):
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty or "Adj Close" not in data:
            raise ValueError(f"No valid data found for {ticker}")
        
        close = data["Adj Close"]
        split_index = int(len(close) * train_ratio)
        
        price_scale = 1000 if close.mean() < 0.1 else 1
        scaled_close = close * price_scale
        
        returns = scaled_close.pct_change().dropna() if close.mean() > 0.1 else np.log(scaled_close / scaled_close.shift(1)).dropna()
        
        train_returns = returns.iloc[:split_index]
        test_returns = returns.iloc[split_index:]
        
        scaler = StandardScaler()
        scaler.fit(train_returns.values.reshape(-1, 1))
        
        train_std = scaler.transform(train_returns.values.reshape(-1, 1))
        test_std = scaler.transform(test_returns.values.reshape(-1, 1))
        
        std_returns = np.concatenate([train_std, test_std]).flatten()
        
        return close[returns.index], pd.Series(std_returns, index=returns.index), split_index
    
    except Exception as e:
        raise RuntimeError(f"Error processing {ticker}: {str(e)}")

def generate_signals(returns, split_index, epochs=50):
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    kernel = Matern32()
    gpr = GPR(data=(X_train, y_train), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    
    trend_train = gpr.predict_f(X_train)[0].numpy().flatten()
    trend_test = gpr.predict_f(X_test)[0].numpy().flatten()
    trend = np.concatenate([trend_train, trend_test])
    
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    train_features = features[:split_index]
    train_labels = y[:split_index].flatten()
    
    model = Sequential([
        Input(shape=(1, 2)),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dense(1, activation="tanh")
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    model.fit(
        train_features.reshape((train_features.shape[0], 1, train_features.shape[1])),
        train_labels,
        epochs=epochs,
        batch_size=32,
        verbose=0
    )
    
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features).flatten()
    return np.clip(predicted_signals, -1, 1)

def backtest_strategy(close, positions):
    entries = positions > 0.2
    exits = positions < -0.2
    
    pf = vbt.Portfolio.from_signals(
        close=close,
        entries=entries,
        exits=exits,
        size=np.abs(positions),
        freq="1D"
    )
    return pf

def main():
    ticker = "BTC-USD"
    print(f"\nBacktesting {ticker}...")
    try:
        close, returns, split_index = fetch_and_process_data(ticker)
        positions = generate_signals(returns, split_index)
        portfolio = backtest_strategy(close, positions)
        
        print(f"Total Return: {portfolio.total_return():.2%}")
        print(f"Sharpe Ratio: {portfolio.sharpe_ratio():.2f}")
        print(f"Max Drawdown: {portfolio.max_drawdown():.2%}")
        
        summary = {
            'Total Return': f"{portfolio.total_return():.2%}",
            'Sharpe Ratio': f"{portfolio.sharpe_ratio():.2f}",
            'Max Drawdown': f"{portfolio.max_drawdown():.2%}"
        }
        
        summary_df = pd.DataFrame([summary])
        print("\nFinal Backtest Results:")
        print(summary_df.to_markdown(tablefmt="grid"))
        
    except Exception as e:
        print(f"Error in backtesting {ticker}: {str(e)}")

if __name__ == "__main__":
    main()



Backtesting BTC-USD...


[*********************100%%**********************]  1 of 1 completed


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Total Return: 57.35%
Sharpe Ratio: 1.45
Max Drawdown: -23.47%

Final Backtest Results:
+----+----------------+----------------+----------------+
|    | Total Return   |   Sharpe Ratio | Max Drawdown   |
+====+================+================+================+
|  0 | 57.35%         |           1.45 | -23.47%        |
+----+----------------+----------------+----------------+
